In [1]:
import os
os.chdir("../../")
import numpy as np
from pandas import DataFrame
from backtesting import Backtest
from backtesting.lib import Strategy

from backtesting.test import EURUSD
from strategies.ichimoku_v1 import IchimokuCloud

/Users/pimpmasterguapo/Documents/github/personal_projects/ai-trading-app/.venv/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/Users/pimpmasterguapo/Documents/github/personal_projects/ai-trading-app/.venv/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [2]:
class IchimokuCloudBacktest(Strategy):
    def init(self):
        self.position_open = False
        self.data.df.reset_index(inplace=True, names="Datetime")
        self.sl_pips = 200

    def calculate_risk_reward_ratio(
        self, entry_price, stop_loss_price, take_profit_price
    ):
        risk_reward_ratio = (take_profit_price - entry_price) / (
            entry_price - stop_loss_price
        )
        return risk_reward_ratio

    def calculate_take_profit(
        self, entry_price, stop_loss_price, risk_reward_ratio, is_buy=True
    ):
        direction_factor = 1 if is_buy else -1
        take_profit_price = entry_price + direction_factor * risk_reward_ratio * abs(
            entry_price - stop_loss_price
        )
        return take_profit_price

    def calculate_pip_amount(self, entry_price, exit_price, pip_factor=0.0001):
        pip_amount = (exit_price - entry_price) / pip_factor
        return pip_amount

    def calculate_new_price(
        self, original_price, pip_value, pip_factor=0.0001, add_pips=True
    ):
        direction = 1 if add_pips else -1
        new_price = original_price + direction * pip_value * pip_factor
        return new_price

    def generate_mandatory_signals(
        self, price_cloud_cross_signal, kijun_sen_direction_signal
    ):
        mandatory_signals = np.sum(
            [price_cloud_cross_signal, kijun_sen_direction_signal]
        )

        return mandatory_signals

    def generate_confirmation_signals(
        self, ichimoku: IchimokuCloud, ichimoku_df: DataFrame
    ):
        (
            senkou_span_cloud_signal,
            senkou_span_cloud_grade,
        ) = ichimoku.senkou_span_cloud_color(ichimoku_df)
        (
            price_kijun_cross_signal,
            price_kijun_cross_grade,
        ) = ichimoku.price_kijun_sen_cross(ichimoku_df)
        (
            tenken_kijun_cross_signal,
            tenken_kijun_cross_grade,
        ) = ichimoku.tenkan_sen_kijun_sen_cross(ichimoku_df)
        (
            price_chikou_cross_signal,
            price_chikou_cross_grade,
        ) = ichimoku.price_chikou_span_cross(ichimoku_df)
        confirmation_grades = np.sum(
            [
                senkou_span_cloud_grade,
                price_kijun_cross_grade,
                tenken_kijun_cross_grade,
                price_chikou_cross_grade,
            ]
        )
        if confirmation_grades >= 10:
            confirmation_signals = np.sum(
                [
                    senkou_span_cloud_signal,
                    price_kijun_cross_signal,
                    tenken_kijun_cross_signal,
                    price_chikou_cross_signal,
                ]
            )

            return confirmation_signals
        else:
            return 0

    def next(self):
        if self.position:
            ichimoku = IchimokuCloud(self.data.df)
            ichimoku_df = ichimoku.get_ichimoku_df()
            price_cloud_cross_signal = ichimoku.price_senkou_span_cross(ichimoku_df)
            kijun_sen_direction_signal = ichimoku.kijun_sen_direction(ichimoku_df)

            mandatory_signals = self.generate_mandatory_signals(
                price_cloud_cross_signal, kijun_sen_direction_signal
            )

            confirmation_signals = self.generate_confirmation_signals(
                ichimoku, ichimoku_df
            )

            if self.position.is_long:
                if mandatory_signals == -2 and confirmation_signals <= -2:
                    self.position.close()
            elif self.position.is_short:
                if mandatory_signals == 2 and confirmation_signals >= 2:
                    self.position.close()
        else:
            ichimoku = IchimokuCloud(self.data.df)
            ichimoku_df = ichimoku.get_ichimoku_df()
            price_cloud_cross_signal = ichimoku.price_senkou_span_cross(ichimoku_df)
            kijun_sen_direction_signal = ichimoku.kijun_sen_direction(ichimoku_df)

            mandatory_signals = self.generate_mandatory_signals(
                price_cloud_cross_signal, kijun_sen_direction_signal
            )

            confirmation_signals = self.generate_confirmation_signals(
                ichimoku, ichimoku_df
            )

            if mandatory_signals == 2 or mandatory_signals == -2:
                if mandatory_signals == 2 and confirmation_signals >= 3:
                    last_close_index = ichimoku_df["Close"].last_valid_index()
                    if last_close_index != None:
                        close = self.data.df.iloc[-1]["Close"]
                        senkou_span_a = ichimoku_df.iloc[last_close_index][
                            "senkou_span_a"
                        ]
                        senkou_span_b = ichimoku_df.iloc[last_close_index][
                            "senkou_span_b"
                        ]
                        senkou_line = np.max([senkou_span_a, senkou_span_b])
                        senkou_stop_loss = senkou_line - (senkou_line * 0.05)
                        pip_stop_loss = self.calculate_new_price(
                            close, self.sl_pips, add_pips=False
                        )
                        stop_loss = np.max([senkou_stop_loss, pip_stop_loss])
                        take_profit = self.calculate_take_profit(
                            close, stop_loss, 1.5, True
                        )

                        # print(
                        #     f"""BUY -> Close: {close}
                        #         TP: {take_profit} TPPIPS: {self.calculate_pip_amount(close, take_profit)}
                        #         SL: {stop_loss} SLPIPS: {self.calculate_pip_amount(close, stop_loss)}
                        #         RRR: {self.calculate_risk_reward_ratio(close,stop_loss,take_profit)}""".strip()
                        # )

                        self.buy(sl=stop_loss, tp=take_profit)
                        self.position_open = True

                elif mandatory_signals == -2 and confirmation_signals <= -3:
                    last_close_index = ichimoku_df["Close"].last_valid_index()
                    if last_close_index != None:
                        close = self.data.df.iloc[-1]["Close"]
                        senkou_span_a = ichimoku_df.iloc[last_close_index][
                            "senkou_span_a"
                        ]
                        senkou_span_b = ichimoku_df.iloc[last_close_index][
                            "senkou_span_b"
                        ]
                        senkou_line = np.min([senkou_span_a, senkou_span_b])
                        senkou_stop_loss = senkou_line * 1.05
                        pip_stop_loss = self.calculate_new_price(close, self.sl_pips)
                        stop_loss = np.min([senkou_stop_loss, pip_stop_loss])
                        take_profit = self.calculate_take_profit(
                            close, stop_loss, 1.5, False
                        )

                        # print(
                        #     f"""SELL -> Close: {close}
                        #         TP: {take_profit} TPPIPS: {self.calculate_pip_amount(take_profit, close)}
                        #         SL: {stop_loss} SLPIPS: {self.calculate_pip_amount(stop_loss, close)}
                        #         RRR: {self.calculate_risk_reward_ratio(close,stop_loss,take_profit)}""".strip()
                        # )

                        self.sell(sl=stop_loss, tp=take_profit)
                        self.position_open = True

In [3]:
bt = Backtest(EURUSD, IchimokuCloudBacktest,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
# bt.plot()

In [4]:
output

Start                     2017-04-19 09:00:00
End                       2018-02-07 15:00:00
Duration                    294 days 06:00:00
Exposure Time [%]                       11.88
Equity Final [$]                  9850.388419
Equity Peak [$]                  10095.215512
Return [%]                          -1.496116
Buy & Hold Return [%]               14.628937
Return (Ann.) [%]                   -1.502031
Volatility (Ann.) [%]                2.000324
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.644546
Avg. Drawdown [%]                    -1.13089
Max. Drawdown Duration      203 days 15:00:00
Avg. Drawdown Duration       72 days 03:00:00
# Trades                                   11
Win Rate [%]                        36.363636
Best Trade [%]                       0.935644
Worst Trade [%]                     -0.702481
Avg. Trade [%]                    

In [5]:
output["_trades"]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,9185,85,152,1.088683,1.08758,-10.131239,-0.001013,85,152,67
1,8844,810,843,1.129494,1.12156,-70.172541,-0.007025,810,843,33
2,8628,1527,1577,1.149665,1.15169,17.473943,0.001762,1527,1577,50
3,8454,2958,3052,1.175436,1.18066,44.162174,0.004444,2958,3052,94
4,8550,3349,3375,1.167330,1.16230,-43.006500,-0.004309,3349,3375,26
5,8521,3532,3633,1.166268,1.17718,92.982175,0.009356,3532,3633,101
6,-8538,3639,3659,1.174766,1.18005,-45.116841,-0.004498,3639,3659,20
7,-8525,3688,3728,1.171353,1.17862,-61.954585,-0.006204,3688,3728,40
8,-8442,4137,4181,1.175484,1.18282,-61.927811,-0.006241,4137,4181,44
9,8334,4184,4242,1.183362,1.18442,8.817372,0.000894,4184,4242,58
